<a href="https://colab.research.google.com/github/aleynakirmizi/st-dbscan/blob/main/ST_DBSCAN_TEST_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [1]:
! pip install ciso8601

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for ciso8601: filename=ciso8601-2.2.0-cp37-cp37m-linux_x86_64.whl size=37527 sha256=00d7c3d59cce213d0307f59bb11d8924c95397a76ea1b4b54b5eddfb6c4f35b1
  Stored in directory: /root/.cache/pip/wheels/ad/25/8f/3b0a82303191efe3c1204f3741c42d8eb2b0236567e22485de
Successfully built ciso8601


In [3]:
import ciso8601
import pandas as pd
import time
import re
import string
from sklearn.feature_extraction.text import  CountVectorizer
import nltk
import numpy as np
import json
from st_dbscan import st_dbSCAN

# **Read Data**

In [4]:
path = '/content/drive/MyDrive/News_Category_Dataset_v2.json'
list_=[]
with open(path) as files:
  for file in files:
    list_.append(json.loads(file))


In [5]:
df = pd.DataFrame(list_).copy()
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [6]:
df.shape

(200853, 6)

In [7]:
df=df.head(1000)


In [8]:
train_data=df[['category','headline','date']]
train_data['date'] = train_data['date'].apply(lambda x: ciso8601.parse_datetime(x))
train_data['date'] = train_data['date'].apply(lambda x: time.mktime(x.timetuple()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# **Cleaning the Data**

In [9]:
nltk.download('punkt')
nltk.download('stopwords')
def clean_text(text):
    text = text.lower()                                  # lower-case all characters
    text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
    text =  re.sub(r'http\S+', '',text)                  # remove urls
    text =  re.sub(r'pic.\S+', '',text)
    text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')   # remove stopwords
    text = " ".join([i for i in words if i not in stopwords and len(i)>2])
    text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
train_data['Text_cleaning'] = train_data.headline.apply(clean_text)
train_data['tokenized_sents'] = train_data.apply(lambda row: nltk.word_tokenize(row['Text_cleaning']), axis=1)
data=train_data.loc[:,['date']].values
df= pd.DataFrame()
df["vector_text"] = train_data['tokenized_sents']
df["date"] = data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
train_data['Text_cleaning'] = train_data.headline.apply(clean_text)
vectorizer = CountVectorizer()
data_vectorizer = vectorizer.fit_transform(train_data['Text_cleaning'])
arr=data_vectorizer.todense()
train_data['vectorized_text']=arr.tolist()
train_data['tokenized_sents'] = train_data.apply(lambda row: nltk.word_tokenize(row['Text_cleaning']), axis=1)
data=train_data.loc[:,['date']].values
df= pd.DataFrame()
df["vector_text"] = train_data['tokenized_sents']
df["date"] = data
data =(data-np.min(data))/(np.max(data)-np.min(data))
df["date"] = data*100


In [36]:
df.head()

,vector_text,date
0,"[mass, shootings, texas, last, week]",100.0
1,"[smith, joins, diplo, nicky, jam, world, cups,...",100.0
2,"[hugh, grant, marries, first, time, age]",100.0
3,"[jim, carrey, blasts, castrato, adam, schiff, ...",100.0
4,"[julianna, margulies, uses, donald, trump, poo...",100.0


# **Train**

In [42]:
st = st_dbSCAN(df,0.8,50,5,5)
ft=st.fit_transform(df)

In [47]:
df.cluster.value_counts()

-1     936
 3      11
 4       7
 7       7
 9       7
 8       7
 5       6
 6       6
 10      6
 1       4
 2       3
Name: cluster, dtype: int64

# **Visualization**

In [48]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(train_data['vectorized_text'].values.tolist())
df["x_component"]=X_embedded[:,0]
df["y_component"]=X_embedded[:,1]
df

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



,vector_text,date,x_component,y_component,cluster
0,"[mass, shootings, texas, last, week]",100.0,-2.035120,-2.209917,-1
1,"[smith, joins, diplo, nicky, jam, world, cups,...",100.0,10.577222,-11.018361,-1
2,"[hugh, grant, marries, first, time, age]",100.0,3.560725,3.808012,-1
3,"[jim, carrey, blasts, castrato, adam, schiff, ...",100.0,-10.898386,21.735212,-1
4,"[julianna, margulies, uses, donald, trump, poo...",100.0,-9.288064,13.342375,-1
...,...,...,...,...,...
995,"[attorney, general, eric, schneiderman, resign...",0.0,15.245600,-18.790941,-1
996,"[sanctuary, cities, working, trump, era, surpr...",0.0,-5.250202,6.582161,-1
997,"[shepard, smith, cant, say, trump, pulled, ira...",0.0,-6.963584,3.231126,8
998,"[stormy, daniels, finds, crunchy, new, way, ch...",0.0,-9.602845,0.872917,-1


In [49]:
X_embedded

array([[ -2.03512  ,  -2.2099168],
       [ 10.577222 , -11.018361 ],
       [  3.5607247,   3.808012 ],
       ...,
       [ -6.9635844,   3.2311258],
       [ -9.602845 ,   0.872917 ],
       [ 12.841851 ,   2.2421627]], dtype=float32)

In [50]:
import plotly.express as px
    
fig = px.scatter(df, x="x_component", y="y_component", color = "cluster", size_max=60)
fig.update_layout(
     height=800)
fig.show()

# **Silhuotte Score**

In [53]:
from sklearn.metrics import silhouette_score
silhouette_score(train_data['vectorized_text'].values.tolist(),df["cluster"].values)

-0.10854254104092846